In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model

In [3]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
EXPERIMENT = 'mh12'
MODEL = 'model_A1'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/n1000/mutate_m1_20_pspec/'
# Models
model_template = f'./models/templates/{MODEL}_noisyor.pl'

Training

In [5]:
train_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_DATA, 
    model_template = model_template, 
    training = 0.8, 
    max_iter = 20, 
    evidence = ['occupancy'],
    facts = ['activity', 'fold_change'],
    suffix = ''
)

Splitting data...
Starting LFI...
problog lfi ./models/mh12/model.pl ./models/mh12/evidence.pl -O ./models/mh12/model_trained.pl --normalize -k ddnnf -v -n 20
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 11.156976
[INFO] Weights to learn: [t(0.33,_fc)::occupancy(P,S,inc), t(0.33,_fc)::occupancy(P,S,dec), t(0.33,_fc)::occupancy(P,S,baseline), t(0.33,E,_dM)::occupancy(P,S,inc), t(0.33,E,_dM)::occupancy(P,S,dec), t(0.33,E,_dM)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(_fc),P,S,inc), lfi_body(1,t(_fc),P,S,dec), lfi_body(2,t(_fc),P,S,baseline), lfi_body(3,t(E,_dM),P,S,inc), lfi_body(4,t(E,_dM),P,S,dec), lfi_body(5,t(E,_dM),P,S,baseline)]
[INFO] Parents: [lfi_par(0,t(_fc),P,S,inc), lfi_par(1,t(_fc),P,S,dec), lfi_par(2,t(_fc),P,S,baseline), lfi_par(3,t(E,_dM),P,S,inc), lfi_par(4,t(E,_dM),P,S,dec), lfi_par(5,t(E,_dM),P,S,baseline)]
[INFO] Initial weights: [0.33, 0.33, 0.33, 0.33, 0.33, 0.33]
[INFO] 
[INFO] Iteration 1
[INFO] Weights after iteration

Testing

In [6]:
# Testing
model_trained = f'{EXP_DIR}model_trained.pl'
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [ ]:
for test_on in test:
    PATH_TO_TESTING = f'{EXP_DIR}{test_on}/'
    compare_to = [test_on, 'truth']
    for compare in compare_to:
        if compare == 'truth':
            path_to_truth = PATH_TO_TRUTH
        else: 
            path_to_truth = PATH_TO_TESTING

        test_results = test_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            path_to_truth = path_to_truth, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'], 
            test_on = test_on, compare = compare,
            mode = 'inference')
        results[f'{test_on}_vs_{compare}'] = test_results

In [8]:
PATH_TO_TESTING = f'{EXP_DIR}testing/'
predictions = predict_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_TESTING, 
    model_trained = model_trained, 
    targets = ['occupancy', 'modification'], 
    mode = 'inference', infer_from='facts', 
    suffix = '_testing')
results['predictions'] = predictions

Inferring data (s955)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s955)...
Inferring data (s588)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s588)...
Inferring data (s942)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s942)...
Inferring data (s339)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s339)...
Inferring data (s354)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s354)...
Inferring data (s437)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finished inference (s437)...
Inferring data (s931)...
problog ./models/mh12/make_predictions.pl -o ./models/mh12/predictions/inferred_example.txt
Finishe

In [9]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)